# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9a6183b700>
├── 'a' --> tensor([[ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353]])
└── 'x' --> <FastTreeValue 0x7f9a6183bac0>
    └── 'c' --> tensor([[ 1.7552, -1.5396,  0.1076,  0.0136],
                        [-1.4201,  2.1441,  0.2391,  0.8347],
                        [ 0.7580,  0.7685,  0.6429, -0.8319]])

In [4]:
t.a

tensor([[ 1.5342,  0.2735, -0.3676],
        [ 0.1177, -1.0617,  1.2353]])

In [5]:
%timeit t.a

64.5 ns ± 0.0593 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9a6183b700>
├── 'a' --> tensor([[ 0.7542, -0.3639,  0.5590],
│                   [ 0.5080,  0.4198, -0.8018]])
└── 'x' --> <FastTreeValue 0x7f9a6183bac0>
    └── 'c' --> tensor([[ 1.7552, -1.5396,  0.1076,  0.0136],
                        [-1.4201,  2.1441,  0.2391,  0.8347],
                        [ 0.7580,  0.7685,  0.6429, -0.8319]])

In [7]:
%timeit t.a = new_value

68.2 ns ± 0.0291 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5342,  0.2735, -0.3676],
               [ 0.1177, -1.0617,  1.2353]]),
    x: Batch(
           c: tensor([[ 1.7552, -1.5396,  0.1076,  0.0136],
                      [-1.4201,  2.1441,  0.2391,  0.8347],
                      [ 0.7580,  0.7685,  0.6429, -0.8319]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5342,  0.2735, -0.3676],
        [ 0.1177, -1.0617,  1.2353]])

In [11]:
%timeit b.a

58 ns ± 0.0874 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4709, -0.0561, -1.8106],
               [ 0.0521,  0.8461,  0.7153]]),
    x: Batch(
           c: tensor([[ 1.7552, -1.5396,  0.1076,  0.0136],
                      [-1.4201,  2.1441,  0.2391,  0.8347],
                      [ 0.7580,  0.7685,  0.6429, -0.8319]]),
       ),
)

In [13]:
%timeit b.a = new_value

499 ns ± 0.179 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

814 ns ± 0.324 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.5 µs ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 688 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 328 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9a61841ca0>
├── 'a' --> tensor([[[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]],
│           
│                   [[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]],
│           
│                   [[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]],
│           
│                   [[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]],
│           
│                   [[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]],
│           
│                   [[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]],
│           
│                   [[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]],
│           
│                   [[ 1.5342,  0.2735, -0.3676],
│                    [ 0.1177, -1.0617,  1.2353]]])
└── 'x' --> <FastTreeValue 0x7f9a5c7b0eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 72 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f998f3a4f10>
├── 'a' --> tensor([[ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353],
│                   [ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353],
│                   [ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353],
│                   [ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353],
│                   [ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353],
│                   [ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353],
│                   [ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353],
│                   [ 1.5342,  0.2735, -0.3676],
│                   [ 0.1177, -1.0617,  1.2353]])
└── 'x' --> <FastTreeValue 0x7f998f3a49d0>
    └── 'c' --> tensor([[ 1.7552, -1.5396,  0.1076,  0.0136],
                        [-1.4201,  2.1441,  0.2391,  0.8347],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 38.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.3 µs ± 68.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.7552, -1.5396,  0.1076,  0.0136],
                       [-1.4201,  2.1441,  0.2391,  0.8347],
                       [ 0.7580,  0.7685,  0.6429, -0.8319]],
              
                      [[ 1.7552, -1.5396,  0.1076,  0.0136],
                       [-1.4201,  2.1441,  0.2391,  0.8347],
                       [ 0.7580,  0.7685,  0.6429, -0.8319]],
              
                      [[ 1.7552, -1.5396,  0.1076,  0.0136],
                       [-1.4201,  2.1441,  0.2391,  0.8347],
                       [ 0.7580,  0.7685,  0.6429, -0.8319]],
              
                      [[ 1.7552, -1.5396,  0.1076,  0.0136],
                       [-1.4201,  2.1441,  0.2391,  0.8347],
                       [ 0.7580,  0.7685,  0.6429, -0.8319]],
              
                      [[ 1.7552, -1.5396,  0.1076,  0.0136],
                       [-1.4201,  2.1441,  0.2391,  0.8347],
                       [ 0.7580,  0.7685,  0.6429, -0.8319]],

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 71.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.7552, -1.5396,  0.1076,  0.0136],
                      [-1.4201,  2.1441,  0.2391,  0.8347],
                      [ 0.7580,  0.7685,  0.6429, -0.8319],
                      [ 1.7552, -1.5396,  0.1076,  0.0136],
                      [-1.4201,  2.1441,  0.2391,  0.8347],
                      [ 0.7580,  0.7685,  0.6429, -0.8319],
                      [ 1.7552, -1.5396,  0.1076,  0.0136],
                      [-1.4201,  2.1441,  0.2391,  0.8347],
                      [ 0.7580,  0.7685,  0.6429, -0.8319],
                      [ 1.7552, -1.5396,  0.1076,  0.0136],
                      [-1.4201,  2.1441,  0.2391,  0.8347],
                      [ 0.7580,  0.7685,  0.6429, -0.8319],
                      [ 1.7552, -1.5396,  0.1076,  0.0136],
                      [-1.4201,  2.1441,  0.2391,  0.8347],
                      [ 0.7580,  0.7685,  0.6429, -0.8319],
                      [ 1.7552, -1.5396,  0.1076,  0.0136],
                   

In [28]:
%timeit Batch.cat(batches)

147 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

349 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
